In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [29]:
out='../output/'
inb='../input/flows/'
pref='idm'
inp = inb + pref + '/'

In [30]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2021-04-01,94,MOZN_EDM,0,ZIMA
1,2021-04-01,94,ZIMA,0,BOT
2,2021-04-01,47,MOZN_EDM,1,ZIMA
3,2021-04-01,47,ZIMA,1,BOT
4,2021-04-01,56,MOZN_EDM,2,ZIMA
...,...,...,...,...,...
377,2022-06-30,36,MOZN_EDM,21,ZIMA
378,2022-06-30,36,MOZN_HCB,21,MOZN_EDM
379,2022-06-30,36,ZIMA,21,BOT
380,2022-06-30,85,MOZS,23,RSAN


In [31]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

price  volume  year  month  day  day_of_week     value  cat
date       hour                                                              
2017-05-04 4       0.40     0.5  2017      5    4            3      0.20  off
           5       0.00     0.0  2017      5    4            3      0.00  off
           6       0.00     0.0  2017      5    4            3      0.00  off
           7       0.00     0.0  2017      5    4            3      0.00  off
           8       1.50     8.0  2017      5    4            3     12.00  off
...                 ...     ...   ...    ...  ...          ...       ...  ...
2022-06-27 19    315.00    50.0  2022      6   27            0  15750.00  off
           20    107.50    97.0  2022      6   27            0  10427.50  off
           21     87.50   151.0  2022      6   27            0  13212.50  off
           22     52.50    96.0  2022      6   27            0   5040.00  off
           23     49.75   149.0  2022      6   27            0   7412.75  off

[43656 rows x 8 columns]

In [32]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2021-04-01,4,65,MOZN_EDM,ZIMA,2021,4,1,3,30.00,1950.00
1,2021-04-01,4,65,ZIMA,BOT,2021,4,1,3,30.00,1950.00
2,2021-04-01,5,54,MOZN_EDM,ZIMA,2021,4,1,3,30.00,1620.00
3,2021-04-01,5,54,ZIMA,BOT,2021,4,1,3,30.00,1620.00
4,2021-04-01,7,74,MOZS,RSAN,2021,4,1,3,75.00,5550.00
...,...,...,...,...,...,...,...,...,...,...,...
7120,2022-06-27,23,60,MOZN_EDM,ZIMA,2022,6,27,0,49.75,2985.00
7121,2022-06-27,23,60,MOZN_HCB,MOZN_EDM,2022,6,27,0,49.75,2985.00
7122,2022-06-27,23,89,MOZS,RSAN,2022,6,27,0,49.75,4427.75
7123,2022-06-27,23,89,RSAN,BOT,2022,6,27,0,49.75,4427.75


In [33]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2021-04-01,4,65,1950.00,30.00,ZIMA,2021,4,1,3
1,2021-04-01,4,65,1950.00,30.00,BOT,2021,4,1,3
2,2021-04-01,5,54,1620.00,30.00,ZIMA,2021,4,1,3
3,2021-04-01,5,54,1620.00,30.00,BOT,2021,4,1,3
4,2021-04-01,7,74,5550.00,75.00,RSAN,2021,4,1,3
...,...,...,...,...,...,...,...,...,...,...
7120,2022-06-27,23,-60,-2985.00,49.75,MOZN_EDM,2022,6,27,0
7121,2022-06-27,23,-60,-2985.00,49.75,MOZN_HCB,2022,6,27,0
7122,2022-06-27,23,-89,-4427.75,49.75,MOZS,2022,6,27,0
7123,2022-06-27,23,-89,-4427.75,49.75,RSAN,2022,6,27,0


In [34]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [35]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week  index  flow    value  \
date       hour                                                             
2021-04-01 4     BOT  2021      4    1            3      1    65  1950.00   
           4     MOZ  2021      4    1            3      0   -65 -1950.00   
           5     BOT  2021      4    1            3      3    54  1620.00   
           5     MOZ  2021      4    1            3      2   -54 -1620.00   
           7     BOT  2021      4    1            3      5    74  5550.00   
...              ...   ...    ...  ...          ...    ...   ...      ...   
2022-06-27 22    HCB  2022      6   27            0   7116   -50 -2625.00   
           22    MOZ  2022      6   27            0  21348   -46 -2415.00   
           23    BOT  2022      6   27            0  14247   149  7412.75   
           23    HCB  2022      6   27            0   7121   -60 -2985.00   
           23    MOZ  2022      6   27            0  21363   -89 -4427.75   

                  price  cat  
date       hour               
2021-04-01 4      30.00  off  
           4      30.00  off  
           5      30.00  off  
           5      30.00  off  
           7      75.00  off  
...                 ...  ...  
2022-06-27 22     52.50  off  
           22    157.50  off  
           23     99.50  off  
           23     49.75  off  
           23    149.25  off  

[7424 rows x 10 columns]

In [36]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat side                                       
2021 4     BOT  off consume      8109  509126.53         159   
           ESW  off consume       186    6231.60           6   
           LES  off supply         -5    -300.00           1   
           MOZ  off supply      -4960 -333726.30         121   
           NAM  off supply       -591  -63619.70          20   
...                               ...        ...         ...   
2022 6     HCB  off supply       -720  -95259.00          11   
           LES  off consume       323   51890.00          11   
           MOZ  off supply      -5786 -515712.32         157   
           ZAMZ off consume        50    5000.00           1   
                    supply        -30   -6000.00           1   

                             price_weight_mean  price_mean      power  
year month node cat side                                               
2021 4     BOT  off consume          62.785366   66.815472  51.000000  
           ESW  off consume          33.503226   33.493333  31.000000  
           LES  off supply           60.000000   60.000000  -5.000000  
           MOZ  off supply           67.283528   71.157107 -40.991736  
           NAM  off supply          107.647547  102.023500 -29.550000  
...                                        ...         ...        ...  
2022 6     HCB  off supply          132.304167  142.476364 -65.454545  
           LES  off consume         160.650155  115.909091  29.363636  
           MOZ  off supply           89.131061   82.596306 -36.853503  
           ZAMZ off consume         100.000000  100.000000  50.000000  
                    supply          200.000000  200.000000 -30.000000  

[107 rows x 6 columns]

In [37]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')